First, add the dataset that contains the pre-trained model and the installation for detectron2:

Then you can proceed to install it via pip and offline installation:

In [1]:
!python3 -m pip install ../input/nflimpactdetectron/detectron2/detectron2 --no-index --find-links ../input/nflimpactdetectron/wheelz/wheelz

In [2]:
# importing all needed libs:

import cv2
import numpy as np 
import pandas as pd
import os
import subprocess

import matplotlib.pyplot as plt
import matplotlib.patches as patches
%matplotlib inline
plt.rcParams['figure.dpi'] = 150

import seaborn as sns

from IPython.display import Video, display
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog
from detectron2.engine import DefaultPredictor
from detectron2.structures import Instances, BoxMode
from detectron2.utils.visualizer import Visualizer


In [3]:
# this basically creates the detectron2 configuration for our pre-trained model
def new_model_cfg():
    cfg = get_cfg()
    model = "COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"
    cfg.merge_from_file(model_zoo.get_config_file(model))
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(model)
    cfg.MODEL.MASK_ON = False
    cfg.INPUT.RANDOM_FLIP = "none"
    cfg.OUTPUT_DIR = "output"
    cfg.DATASETS.TRAIN = ("nflimpact_train",)
    cfg.DATASETS.TEST = ("nflimpact_test",)

    cfg.DATALOADER.NUM_WORKERS = 2
    cfg.SOLVER.IMS_PER_BATCH = 4
    cfg.SOLVER.CHECKPOINT_PERIOD = 2000
    cfg.SOLVER.STEPS = (21000, 50000)
    cfg.SOLVER.MAX_ITER = 200000
    cfg.SOLVER.BASE_LR = 0.001
    cfg.TEST.EVAL_PERIOD = 2000
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(classes)
    cfg.DATALOADER.FILTER_EMPTY_ANNOTATIONS = True
    return cfg

In [4]:
# this sets up the pre-trained model with the weights from our dataset and creates a predictor off of it
classes = ["helmet"]
MetadataCatalog.get("nflimpact").set(thing_classes=classes)
cfg = new_model_cfg()
cfg.MODEL.WEIGHTS = "../input/nflimpactdetectron/model_v2_0071999.pth"
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(classes)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
predictor = DefaultPredictor(cfg)

In [5]:
# some sample image from the training set
img_path = '/kaggle/input/nfl-impact-detection/images/57503_000116_Endzone_frame443.jpg'
img = cv2.imread(img_path) 
plt.imshow(img)
plt.show()

In [6]:
# predict on the image
model_output = predictor(img)

model_output = model_output["instances"].to("cpu")
# only include detected instances with high enough score
ni = model_output[model_output.scores > 0.9]

# use the built-in visualizer to draw the boxes onto the image
v = Visualizer(img[:, :, ::-1], metadata=MetadataCatalog.get("nflimpact"))
img = v.draw_instance_predictions(ni).get_image()
plt.imshow(img)
plt.show()